In [2]:
import numpy as np
import math
import sys,os
import xarray as xr

from netCDF4 import Dataset, default_fillvals
from datetime import datetime

import matplotlib.pyplot as plt

In [4]:
## Load utility library

# IPython extension to reload modules before executing user code.
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload

%aimport libutil

# Reload all modules imported with %aimport every time before executing the Python code typed.
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Coupler fields may be empty at initialization

Leo wrote to BillS: 
> (2) There still is an unexplained difference between GrIS extent in FV1 and FV2. According to the documentation, this should not be the case: Over the CISM domain (typically Greenland in CESM2), CISM dictates glacier areas and topographic elevations, overriding the values on CLM’s surface dataset. I am unsure how to proceed at this one. It might have to do with me using a wrong definition of “ice extent” in CLM. In the coupler history snapshot that I have, all fields like “x2l_Sg_icemask” and “x2l_Sg_ice_covered01” are zeroed out, even though “g2x_Sg_ice_covered” is valid. I can imagine this is done because these fields are only relevant at the start of the simulation, when one-way coupled at least. Would you have suggestions on how to obtain these coupler fields? 

BillS wrote to Leo:

> Regarding the missing coupler history fields: these should be present and valid every time step (**but maybe not in initialization**), so I'm not sure why you're seeing this. Do you want to point me to a particular cpl hist file showing this issue?

First, I show that Bill is right here: 

In [5]:
datadir = "/glade/u/home/lvank/work/projects/reconcile_smb/archive/b.e21.BHIST.f09_g17.CMIP6-historical.001_normF/"

In [6]:
filename1 = "cpl/hist/b.e21.BHIST.f09_g17.CMIP6-historical.001_normF.cpl.hi.1850-01-01-00000.nc" # INITIALIZATION
filename2 = "cpl/hist/b.e21.BHIST.f09_g17.CMIP6-historical.001_normF.cpl.hi.1850-01-02-00000.nc" # AFTER DAY 1

In [7]:
ds1 = xr.open_dataset(os.path.join(datadir, filename1))
ds2 = xr.open_dataset(os.path.join(datadir, filename2))

In [8]:
icemask1 = ds1['x2l_Sg_icemask'][0].to_masked_array()
print('init:', np.unique(icemask1).count())

init: 1


In [9]:
icemask2 = ds2['x2l_Sg_icemask'][0].to_masked_array()
print('after day 1:', np.unique(icemask2).count())

after day 1: 348


# 2. Comparing coupler fields across different grid resolutions
Using library functions for uniformity.

## Ice sheet grid mask (`Sg_icemask`) 
In order of decreasing resolution

In [25]:
# FV 1
libutil.print_CPL_icemask_areas(ds2)

CPL-G, : 2191143.1616609427
CPL-L, : 2188117.058060941


In FV1, icemask on the global side is **smaller**:

In [20]:
2188117.1 / 2191143.2

0.9986189401039604

In [23]:
# FV2 coupler file
filename = "/gpfs/fs1/work/lvank/projects/reconcile_smb/archive/I2000_FV2_test/cpl/hist/I2000_FV2_test.cpl.hi.0001-01-02-00000.nc"
libutil.print_CPL_icemask_areas(xr.open_dataset(filename, decode_times=False))

CPL-G, : 2191143.1616609427
CPL-L, : 2194129.3110708212


In FV2, icemask on the global side is **bigger**:

In [24]:
2194129.3/2191143.2

1.0013628045852958

In [21]:
# T31 coupler file
filename = "/glade/u/home/lvank/work/projects/reconcile_smb/archive/I2000_T31_test/cpl/hist/I2000_T31_test.cpl.ha.0002.nc"
libutil.print_CPL_icemask_areas(xr.open_dataset(filename, decode_times=False), 
                                fldname_xg='g2xavg_Sg_icemask', fldname_xl='x2lavg_Sg_icemask')

CPL-G, : 2191143.1616609427
CPL-L, : 2212744.636371046


In T31, icemask on the global side is **bigger**:

In [22]:
2212744.6/2191143.2

1.0098585067374874

## Glacier cover (`Sg_ice_covered`)
Using library function for uniformity

In [30]:
libutil.print_CPL_glaciercover_areas(ds2)

CPL-G, : 1812254.1880229723
CPL-L, : 1908004.055097825


In [33]:
# FV2 coupler file
filename = "/gpfs/fs1/work/lvank/projects/reconcile_smb/archive/I2000_FV2_test/cpl/hist/I2000_FV2_test.cpl.hi.0001-01-02-00000.nc"
libutil.print_CPL_glaciercover_areas(xr.open_dataset(filename, decode_times=False))

CPL-G, : 1812254.1880229723
CPL-L, : 2037214.650930137


In [32]:
# T31 coupler file
filename = "/glade/u/home/lvank/work/projects/reconcile_smb/archive/I2000_T31_test/cpl/hist/I2000_T31_test.cpl.ha.0002.nc"
libutil.print_CPL_glaciercover_areas(xr.open_dataset(filename, decode_times=False), 
                                fldname_xg='g2xavg_Sg_ice_covered', fldname_xl='x2lavg_Sg_ice_covered')

CPL-G, : 1812254.1880229723
CPL-L, : 2321410.824828118
